In [1]:
import torch
from torch import nn

In [2]:
torch.manual_seed(1)
rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True)

w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0

print('W_xh shape:', w_xh.shape)
print('W_hh shape:', w_hh.shape)
print('b_xh shape:', b_xh.shape)
print('b_hh shape:', b_hh.shape)

#INput of this layer is (batch size, sequence_length, feature size)

W_xh shape: torch.Size([2, 5])
W_hh shape: torch.Size([2, 2])
b_xh shape: torch.Size([2])
b_hh shape: torch.Size([2])


### Manually computing RNN

In [3]:
x_seq = torch.tensor([[1.]*5, [2.]*5, [3.]*5]).float()
print('Input data:',x_seq)

##output of the simple RNN:
output, hn = rnn_layer(torch.reshape(x_seq, (1, 3, 5)))

##manually computing the output
out_man = []
for t in range(3):
    xt = torch.reshape(x_seq[t], (1,5))
    print(f"Time Step {t} =>")
    print('     Input       :', xt.numpy())

    ht = torch.matmul(xt, torch.transpose(w_xh, 0, 1)) + b_xh
    print('     Hidden      :', ht.detach().numpy())

    if t>0:
        prev_h = out_man[t-1]
    else:
        prev_h = torch.zeros((ht.shape))

    #RNN
    ot = ht + torch.matmul(prev_h, torch.transpose(w_hh, 0, 1)) + b_hh
    ot = torch.tanh(ot)
    out_man.append(ot)
    print('   Output (manual) :', ot.detach().numpy())
    print('   RNN output      :', output[:, t].detach().numpy())
    print()

Input data: tensor([[1., 1., 1., 1., 1.],
        [2., 2., 2., 2., 2.],
        [3., 3., 3., 3., 3.]])
Time Step 0 =>
     Input       : [[1. 1. 1. 1. 1.]]
     Hidden      : [[-0.4701929  0.5863904]]
   Output (manual) : [[-0.3519801   0.52525216]]
   RNN output      : [[-0.3519801   0.52525216]]

Time Step 1 =>
     Input       : [[2. 2. 2. 2. 2.]]
     Hidden      : [[-0.88883156  1.2364397 ]]
   Output (manual) : [[-0.68424344  0.76074266]]
   RNN output      : [[-0.68424344  0.76074266]]

Time Step 2 =>
     Input       : [[3. 3. 3. 3. 3.]]
     Hidden      : [[-1.3074701  1.886489 ]]
   Output (manual) : [[-0.8649416   0.90466356]]
   RNN output      : [[-0.8649416   0.90466356]]



## Sentiment Analysis of IMDb movie reviews

### Downloading Dataset

In [4]:
!pip install torchtext torchdata -q

In [5]:
from torchtext.datasets import IMDB

In [6]:
path = './data/IMDB/'
train_dataset = IMDB(split='train', root=path)
test_dataset = IMDB(split='test', root=path)

### Preprocessing 

1. Split the training dataset into training and validation
2. Identify unique words in training set
3. Map each unique word to a unique integer and encode the review text into encoded integers
4. Divide the dataset into mini-batches as input to the model

In [7]:
# label, text = list(train_dataset)[0]
# print(label)
# print('\n\n\n',text)

In [8]:
## 1. Split the datasets
from torch.utils.data.dataset import random_split

torch.manual_seed(1)
train_dataset, valid_dataset = random_split(list(train_dataset), [20000, 5000])

In [9]:
## 2. Filtering and removing punctuation and non-letter character & finding unique word
import re
from collections import Counter, OrderedDict

token_counts = Counter()

def tokenizer(text):
    text = re.sub('<[^>]*>', '', text)
    emoticons = re.findall('(?::|;|=)(?:-)?(?:\)|\(|D|P)', text.lower())
    text = re.sub('[\W]+', ' ', text.lower()) +\
        ' '.join(emoticons).replace('-', '')
    tokenized = text.split()
    return tokenized

for label, line in train_dataset:
    tokens = tokenizer(line)
    token_counts.update(tokens)
 
    
print('Vocab-size:', len(token_counts))

Vocab-size: 69023


In [10]:
## 3. Map unique word to a unique integer
from torchtext.vocab import vocab 

#Sort the items to the most frequent 
sorted_by_freq_tuples = sorted(
    token_counts.items(), key=lambda x: x[1], reverse=True
)

ordered_dict = OrderedDict(sorted_by_freq_tuples)
vocab = vocab(ordered_dict)
#Index 1 is reserved for unknown words, Index 0 serve as a padding token
vocab.insert_token('<pad>', 0)
vocab.insert_token('<unk>', 1)
vocab.set_default_index(1)

In [11]:
#How to use the vocab object
print([vocab[token] for token in ['this', 'is','an','example']])

[11, 7, 35, 457]


In [12]:
# Defining pipeline for transformation
text_pipeline = lambda x: [vocab[token] for token in tokenizer(x)]
label_pipeline = lambda x: 1. if x =='pos' else 0.

In [13]:
device = 'mps'

In [14]:
def collate_batch(batch):
    label_list, text_list, lengths = [], [], []
    for _label, _text in batch:
        label_list.append(label_pipeline(_label))
        processed_text = torch.tensor(text_pipeline(_text), 
                                      dtype=torch.int64)
        text_list.append(processed_text)
        lengths.append(processed_text.size(0))
    label_list = torch.tensor(label_list)
    lengths = torch.tensor(lengths)
    #Pads each sequences with to fixed length
    padded_text_list = nn.utils.rnn.pad_sequence(
        text_list, batch_first=True)
    return padded_text_list.to(device), label_list.to(device), lengths.to(device)

In [15]:
#Example
from torch.utils.data import DataLoader
dataloader = DataLoader(train_dataset, batch_size=4, shuffle=False, collate_fn=collate_batch)
text_batch, label_batch, length_batch = next(iter(dataloader))
print(text_batch)
print(label_batch)
print(length_batch)
print(text_batch.shape)

tensor([[   35,  1739,     7,   449,   721,     6,   301,     4,   787,     9,
             4,    18,    44,     2,  1705,  2460,   186,    25,     7,    24,
           100,  1874,  1739,    25,     7, 34415,  3568,  1103,  7517,   787,
             5,     2,  4991, 12401,    36,     7,   148,   111,   939,     6,
         11598,     2,   172,   135,    62,    25,  3199,  1602,     3,   928,
          1500,     9,     6,  4601,     2,   155,    36,    14,   274,     4,
         42945,     9,  4991,     3,    14, 10296,    34,  3568,     8,    51,
           148,    30,     2,    58,    16,    11,  1893,   125,     6,   420,
          1214,    27, 14542,   940,    11,     7,    29,   951,    18,    17,
         15994,   459,    34,  2480, 15211,  3713,     2,   840,  3200,     9,
          3568,    13,   107,     9,   175,    94,    25,    51, 10297,  1796,
            27,   712,    16,     2,   220,    17,     4,    54,   722,   238,
           395,     2,   787,    32,    27,  5236,  

/opt/miniconda3/envs/pytorch/lib/python3.9/site-packages/torch/_tensor_str.py:115: UserWarning: The operator 'aten::nonzero' is not currently supported on the MPS backend and will fall back to run on the CPU. This may have performance implications. (Triggered internally at /Users/runner/work/_temp/anaconda/conda-bld/pytorch_1670525849783/work/aten/src/ATen/mps/MPSFallback.mm:11.)
  nonzero_finite_vals = torch.masked_select(


In [16]:
batch_size = 32

train_dl = DataLoader(train_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

valid_dl = DataLoader(valid_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

test_dl = DataLoader(test_dataset, batch_size=batch_size, shuffle=False, collate_fn=collate_batch)

In [17]:
# Creating an embedding example

#INput must be [ batch size, input_length (length of sequence) ]
embedding = nn.Embedding(
    #Should include the number of unique words and specials tokens -> padding & <unk>
    num_embeddings=10, #number of unique tokens n+2
    embedding_dim=3, #embedding output size each token
    padding_idx=0 # token used for padding
)
#Output [batch_size, input_length, embedding_dim]

#A batch of 2 samples of 4 indices each
text_encoded_input = torch.LongTensor([
    [1,2,4,5],
    [4,3,2,0]
])

print(embedding(text_encoded_input))
print(embedding(text_encoded_input).shape)

tensor([[[ 0.7039, -0.8321, -0.4651],
         [-0.3203,  2.2408,  0.5566],
         [-0.4643,  0.3046,  0.7046],
         [-0.7106, -0.2959,  0.8356]],

        [[-0.4643,  0.3046,  0.7046],
         [ 0.0946, -0.3531,  0.9124],
         [-0.3203,  2.2408,  0.5566],
         [ 0.0000,  0.0000,  0.0000]]], grad_fn=<EmbeddingBackward0>)
torch.Size([2, 4, 3])


### Building the model

In [18]:
class RNN(nn.Module):

    def __init__(self, vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size):
        super().__init__()

        self.embedding = nn.Embedding(num_embeddings=vocab_size, embedding_dim=embed_dim, padding_idx=0)

        self.rnn = nn.LSTM(embed_dim, rnn_hidden_size, batch_first=True)

        #self.gru = nn.GRU(input_size, hidden_size, num_layers, batch_first=True)
        self.fc1 = nn.Linear(rnn_hidden_size, fc_hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(fc_hidden_size, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, text, lengths):
        out = self.embedding(text)
        out = nn.utils.rnn.pack_padded_sequence(out, lengths.cpu().numpy(), enforce_sorted=False, batch_first=True)

        out, (hidden, cell) = self.rnn(out)
        out = hidden[-1, :, :]
        out = self.fc1(out)
        out = self.relu(out)
        out = self.fc2(out)
        out = self.sigmoid(out)
        return out
         
    
vocab_size = len(vocab)
embed_dim = 20
rnn_hidden_size = 64
fc_hidden_size = 64

torch.manual_seed(1)
model = RNN(vocab_size, embed_dim, rnn_hidden_size, fc_hidden_size) 
model = model.to(device)

### Training Loop

In [19]:
def train(dataloader):
    model.train()
    
    total_acc, total_loss = 0, 0

    for text_batch, label_batch, lengths in dataloader:
        optimizer.zero_grad()

        pred = model(text_batch, lengths)[:, 0]
        loss = loss_fn(pred, label_batch)

        loss.backward()
        optimizer.step()
        
        total_acc += ( (pred >= 0.5).float() == label_batch ).float().sum().item()
        total_loss += loss.item() * label_batch.size(0)

    return total_acc / len(dataloader.dataset), total_loss / len(dataloader.dataset)

### Evaluation Loop

In [20]:
def evaluate(dataloader):
    model.eval()

    total_acc, total_loss = 0, 0

    with torch.inference_mode():
        for text_batch, label_batch, lengths in dataloader:
            pred = model(text_batch, lengths)[:, 0]

            loss = loss_fn(pred, label_batch)

        total_acc += ( (pred>=0.5).float() == label_batch ).float().sum().item()
        total_loss += loss.item()*label_batch.size(0)

    return total_acc/len(dataloader.dataset), total_loss/len(dataloader.dataset)

In [21]:
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

num_epochs = 10 

torch.manual_seed(1)
 
for epoch in range(num_epochs):
    acc_train, loss_train = train(train_dl)
    acc_valid, loss_valid = evaluate(valid_dl)
    print(f'Epoch {epoch} accuracy: {acc_train:.4f} val_accuracy: {acc_valid:.4f}')

In [ ]:
acc_test, _ = evaluate(test_dl)
print(f"test_accuracy: {acc_test:.4f}")